<a href="https://www.kaggle.com/code/mdsayeduzzaman/notebookddacd2cfa9?scriptVersionId=104389029" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
print("complete")

/kaggle/input/cvpdfdata/agaspar_cv.pdf
complete


In [2]:
#all pip install
!pip install pdfplumber
print("complete")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 349.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 4.2 MB/s eta 0:00:00
complete


In [3]:
#all import
import pdfplumber
import pandas as pd
import os
import spacy
from spacy.matcher import Matcher

from datetime import datetime

import re
from nltk.corpus import stopwords



print("complete")

complete


In [4]:
# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)
print("complete")

complete


In [5]:
def extract_pdf(pdf_path):
    linesOfFile = []
    with pdfplumber.open(pdf_path) as pdf:
        for pdf_page in pdf.pages:
            single_page_text = pdf_page.extract_text()
            for line in single_page_text.split('\n'):
                linesOfFile.append(line)
                #print(linesOfFile)
    return linesOfFile


#folder_with_pdfs = '../input/cvalltogether'
folder_with_pdfs = '../input/cvpdfdata'


linesOfFiles = []



listOfPdfFiles=[]

for pdf_file in os.listdir(folder_with_pdfs):
    if pdf_file.endswith('.pdf'):
        #print(pdf_file)
        listOfPdfFiles.append(pdf_file)
        pdf_file_path = os.path.join(folder_with_pdfs, pdf_file)
        linesOfFile = extract_pdf(pdf_file_path)
        linesOfFiles.append(linesOfFile)
        
        
        
        
df = pd.DataFrame(linesOfFiles)
#df.to_csv('test.csv')
print("complete")

complete


In [6]:
listOfPdfFiles

['agaspar_cv.pdf']

In [7]:
df

,0,1,2,3,4,5,6,7,8,9,...,264,265,266,267,268,269,270,271,272,273
0,András Gáspár 12/10/2016,Contact StewardObservatory Tel: (520)-360-0983,Information TheUniversityofArizona agaspar@as....,"933NCherryAvenue,Rm278 http://merope.as.arizon...","Tucson,AZ85721-0065 U.S.permanentresident","Research Circumstellar Disks, Exozodi, Extraso...","Areas ters,ISM,Coronagraphy,InfraredDetectors,...","Education 2011 Ph.D.,Astronomy,StewardObservat...",Thesis: “Observationsandmodelsofinfrareddebris...,theirevolution”,...,"∗10. AVariableStarSurveyoftheOpenClusterNGC2126,","Gáspár, A., Kiss, L. L., Derekas, A., Bedding,...","G.M.,andVáradi,M.","2004,CommunicationsinAstroseismology,145,70","11. AVariableStarSurveyoftheOpenClusterNGC2126,","Derekas,A.,Kiss,L.L.,Bedding,T.R.,Gáspár,A.,Sá...","Kaspi,S.,andKiss,Cs.","2004, Variable Stars in the Local Group, IAU C...","Proceedings,Vol.310,p.162",8


In [8]:
# to do : do partial match , i.e. word+etc

reference=[ 
            #node1
            "articles","books","chapters","citations","editorials","journals",
               "seminars","scholarly",
            #node2
            "awards","proposals","grants","honors","scholarships","sponsored",
            #node3
            "appointments","experiences","services","employments","practices",
                "professionals",
            #node4
            "affiliations","memberships","committees",
            #node5
            "contacts","introductions",
            #node6
            "publications","conferences","presentations","newsletters","reports",
            #node7
            "educations","certificates","certifications",
            #node8
            "researches",
            #node9
            "teaching","outreaches"
          ]
#to store data
allCVData=[]
allCVData.append(("fileName","personName","sectionName","head","relation/lebel","tail"))

print(allCVData)
print(reference,len(reference))

print("complete")

[('fileName', 'personName', 'sectionName', 'head', 'relation/lebel', 'tail')]
['articles', 'books', 'chapters', 'citations', 'editorials', 'journals', 'seminars', 'scholarly', 'awards', 'proposals', 'grants', 'honors', 'scholarships', 'sponsored', 'appointments', 'experiences', 'services', 'employments', 'practices', 'professionals', 'affiliations', 'memberships', 'committees', 'contacts', 'introductions', 'publications', 'conferences', 'presentations', 'newsletters', 'reports', 'educations', 'certificates', 'certifications', 'researches', 'teaching', 'outreaches'] 36
complete


In [9]:
def dfPartByPart(textList):
    collectedText=[]
    collectedByDictionary=dict()
    collectedParts=dict()
    keyDictionary="introductions"
    temp=[]



    for smallPartText in textList:
        if smallPartText==None:
            continue
   
        for partOfSmallPartText in smallPartText.split():
            if len(partOfSmallPartText)<2:
                continue
            result = list(filter(lambda x: x.startswith(partOfSmallPartText.lower()), reference))
            if result:
                
                collectedText.append(temp)
                if keyDictionary in collectedByDictionary:
                    collectedByDictionary[keyDictionary]=collectedByDictionary[keyDictionary]+temp
                else:
                    collectedByDictionary[keyDictionary]=temp
                keyDictionary=result[0]
                temp=[]
                break
        
        temp.append(smallPartText)
    
    
    collectedText.append(temp)
    if keyDictionary in collectedByDictionary:
        collectedByDictionary[keyDictionary]=collectedByDictionary[keyDictionary]+temp
    else:
        collectedByDictionary[keyDictionary]=temp

    return collectedByDictionary
print("complete")

complete


In [10]:
matrix_res = df.to_numpy()

print(len(matrix_res))

for i in range(len(matrix_res)):
   
    
    sectionDictionary=dfPartByPart(matrix_res[i])
    
#print(sectionDictionary)
print("complete")

1
complete


In [11]:
print(sectionDictionary["contacts"])
print(sectionDictionary["introductions"])


['Contact StewardObservatory Tel: (520)-360-0983', 'Information TheUniversityofArizona agaspar@as.arizona.edu', '933NCherryAvenue,Rm278 http://merope.as.arizona.edu/˜agaspar', 'Tucson,AZ85721-0065 U.S.permanentresident']
['András Gáspár 12/10/2016', '2004, Variable Stars in the Local Group, IAU Colloquium 193, New Zealand, ASP Conference', 'Proceedings,Vol.310,p.162', '8']


In [12]:
revisedSectionDictionary=dict()

for sectionPart in sectionDictionary:
    if sectionPart in ["articles","books","chapters","citations","editorials","journals","seminars","scholarly"]:
        
        temp="articles / books / chapters / citations / editorials / journals / seminars / scholarly"
        if temp in revisedSectionDictionary:
            
            revisedSectionDictionary[temp].extend(sectionDictionary[sectionPart])
        else:
            revisedSectionDictionary[temp]=sectionDictionary[sectionPart]
    elif sectionPart in ["awards","proposals","grants","honors","scholarships","sponsored"]:
        
        temp="awards / grants / proposals / honors / scholarships / sponsored"
        if temp in revisedSectionDictionary:
            
            revisedSectionDictionary[temp].extend(sectionDictionary[sectionPart])
        else:
            revisedSectionDictionary[temp]=sectionDictionary[sectionPart]
    
    elif sectionPart in ["appointments","experiences","services","employments","practices","professionals"]:
        
        temp="appointments / experiences / services / employments / practices / professionals"
        if temp in revisedSectionDictionary:
            
            revisedSectionDictionary[temp].extend(sectionDictionary[sectionPart])
        else:
            revisedSectionDictionary[temp]=sectionDictionary[sectionPart]
    elif sectionPart in ["affiliations","memberships","committees"]:
        
        temp="affiliations / memberships / committees"
        if temp in revisedSectionDictionary:
            
            revisedSectionDictionary[temp].extend(sectionDictionary[sectionPart])
        else:
            revisedSectionDictionary[temp]=sectionDictionary[sectionPart]
    elif sectionPart in ["contacts","introductions"]:
        
        temp="contacts / introductions"
        if temp in revisedSectionDictionary:
            
            revisedSectionDictionary[temp].extend(sectionDictionary[sectionPart])
        else:
            revisedSectionDictionary[temp]=sectionDictionary[sectionPart]
    elif sectionPart in ["publications","conferences","presentations","newsletters","reports"]:
        
        temp="publications / conferences / presentations / newsletters / reports"
        if temp in revisedSectionDictionary:
            
            revisedSectionDictionary[temp].extend(sectionDictionary[sectionPart])
        else:
            revisedSectionDictionary[temp]=sectionDictionary[sectionPart]
    elif sectionPart in ["educations","certificates","certifications"]:
        
        temp="educations / certificates / certifications"
        if temp in revisedSectionDictionary:
            
            revisedSectionDictionary[temp].extend(sectionDictionary[sectionPart])
        else:
            revisedSectionDictionary[temp]=sectionDictionary[sectionPart]
    elif sectionPart in ["researches"]:
        
        temp="researches"
        if temp in revisedSectionDictionary:
            
            revisedSectionDictionary[temp].extend(sectionDictionary[sectionPart])
        else:
            revisedSectionDictionary[temp]=sectionDictionary[sectionPart]
    elif sectionPart in ["teaching","outreaches"]:
        
        temp="teaching / outreaches"
        if temp in revisedSectionDictionary:
            
            revisedSectionDictionary[temp].extend(sectionDictionary[sectionPart])
        else:
            revisedSectionDictionary[temp]=sectionDictionary[sectionPart]
    
print("complete")

complete


In [13]:
print(sectionDictionary["contacts"])
print(sectionDictionary["introductions"])
print("****************************")


print(revisedSectionDictionary["contacts / introductions"])
print("complete")

['Contact StewardObservatory Tel: (520)-360-0983', 'Information TheUniversityofArizona agaspar@as.arizona.edu', '933NCherryAvenue,Rm278 http://merope.as.arizona.edu/˜agaspar', 'Tucson,AZ85721-0065 U.S.permanentresident']
['András Gáspár 12/10/2016', '2004, Variable Stars in the Local Group, IAU Colloquium 193, New Zealand, ASP Conference', 'Proceedings,Vol.310,p.162', '8', 'Contact StewardObservatory Tel: (520)-360-0983', 'Information TheUniversityofArizona agaspar@as.arizona.edu', '933NCherryAvenue,Rm278 http://merope.as.arizona.edu/˜agaspar', 'Tucson,AZ85721-0065 U.S.permanentresident']
****************************
['András Gáspár 12/10/2016', '2004, Variable Stars in the Local Group, IAU Colloquium 193, New Zealand, ASP Conference', 'Proceedings,Vol.310,p.162', '8', 'Contact StewardObservatory Tel: (520)-360-0983', 'Information TheUniversityofArizona agaspar@as.arizona.edu', '933NCherryAvenue,Rm278 http://merope.as.arizona.edu/˜agaspar', 'Tucson,AZ85721-0065 U.S.permanentresident']


In [14]:
print(revisedSectionDictionary.keys())

dict_keys(['contacts / introductions', 'researches', 'educations / certificates / certifications', 'appointments / experiences / services / employments / practices / professionals', 'awards / grants / proposals / honors / scholarships / sponsored', 'teaching / outreaches', 'publications / conferences / presentations / newsletters / reports'])
